In [ ]:
!pip install nomic  # if first time running, install nomic

In [ ]:
import pandas as pd
import numpy as np
import nomic
from nomic import atlas
import ast
# pd.set_option('max_colwidth', 1000)

In [ ]:
# Run this if you don't have a nomic access token
#!nomic login
# Run this if you have nomic access token
!nomic login 'HfFeBu516-CFtRnOFlEY9k-A5DWMqyKrhNe7XyxSv3irV'

# Load data
n_load = 10000 # how many rows to load. This affects run time
file_path = '../data/documents_202310221252.csv'
df_doc = pd.read_csv(file_path, encoding='utf-8-sig')[:n_load]

# Nomic only allows unique ids
for i in range(len(df_doc)):
    df_doc['id'].iloc[i] = df_doc.iloc[i]['id']+str(i)
print(df_doc['id'].duplicated().sum()) # check for duplicate id

# Nomic only allow ids of length 36 and less
df_doc['id'] = df_doc['id'].apply(lambda x: x[-36:])
print('max id length-',df_doc['id'].str.len().max()) # check max id length

# Turn embedding column from string to numpy array
df_doc['embedding_np'] = df_doc['embedding'].apply(lambda x: ast.literal_eval(x))
numpy_df_doc = np.array(list(df_doc['embedding_np']))
print('numpy_df_doc.shape- ',numpy_df_doc.shape) # check embedding dimension

# Nomic expects dictionary format as input data
df_doc_dict = df_doc[['id','title','subtitle','content','score', 'url']].to_dict('records')

# This line of code will generate a link to webapp visualisation
response = atlas.map_embeddings(embeddings=numpy_df_doc,
                                data=df_doc_dict,
                                id_field='id',
                                colorable_fields=['score'],
                                name="Test example", #change this every new run
                                description="A text map with test embedding.",
                                #build_topic_model=True,  #ignore
                                reset_project_if_exists=True) #checks duplicate project name